In [1]:
from dotenv import load_dotenv, find_dotenv
# read local .env file
_ = load_dotenv(find_dotenv())

In [2]:
from langchain_anthropic import ChatAnthropic
from langchain_core.pydantic_v1 import BaseModel, Field

In [3]:
opus_model_id = 'claude-3-opus-20240229'
sonnet_model_id = 'claude-3-sonnet-20240229'
haiku_model_id = 'claude-3-haiku-20240307'

In [4]:
# Data model
class code(BaseModel):
    """Code output"""

    prefix: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")

In [5]:
llm = ChatAnthropic(model=opus_model_id) #,default_headers={"anthropic-beta": "tools-2024-04-04"}
structured_llm = llm.with_structured_output(code, include_raw=True)

/Users/rosclay/Documents/VSCode/langgraph-multi-agent/venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [ ]:
code_output = structured_llm.invoke(
    "Write a python program that prints the string 'hello world' and tell me how it works in a sentence"
)

In [ ]:
code_output

In [6]:
class GetWeather(BaseModel):
    """Get the current weather in a given location"""

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")


llm_with_tools = llm.bind_tools([GetWeather])

/Users/rosclay/Documents/VSCode/langgraph-multi-agent/venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `bind_tools` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [ ]:
ai_msg = llm_with_tools.invoke(
    "what is the weather like in San Francisco",
)
ai_msg

In [ ]:
ai_msg.content[0]

In [ ]:
ai_msg.content[1]

In [ ]:
from langchain_anthropic.output_parsers import ToolsOutputParser

In [ ]:
parser = ToolsOutputParser()
chain = llm_with_tools #| parser
result = chain.invoke("What is the weather like in nyc, la, sf and cleveland")

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

In [ ]:
result.content

In [ ]:
parser = ToolsOutputParser()
chain = llm_with_tools | parser
result = chain.invoke("What is the weather like in nyc, la, sf and cleveland")

In [ ]:
result

In [ ]:
structured_llm = llm.with_structured_output(GetWeather)
structured_llm.invoke(
    "what is the weather like in San Francisco",
)

In [8]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

messages = [
    HumanMessage("What is the weather like in San Francisco"),
    AIMessage(
        content=[
            {
                "text": '<thinking>\nBased on the user\'s question, the relevant function to call is GetWeather, which requires the "location" parameter.\n\nThe user has directly specified the location as "San Francisco". Since San Francisco is a well known city, I can reasonably infer they mean San Francisco, CA without needing the state specified.\n\nAll the required parameters are provided, so I can proceed with the API call.\n</thinking>',
                "type": "text",
            },
            {
                "type": "tool_use",
                "id": "toolu_01SCgExKzQ7eqSkMHfygvYuu",
                "name": "GetWeather",
                "input": {"location": "San Francisco, CA"},
                "text": None,
            },
        ],
    ),
    ToolMessage(
        "Rain. High 54F. Winds SW at 15 to 25 mph. Chance of rain 100%.",
        tool_call_id="toolu_01SCgExKzQ7eqSkMHfygvYuu",
    ),
]
llm_with_tools.invoke(messages)




AIMessage(content='According to the weather report, it is currently raining in San Francisco, with a high temperature of 54°F (12°C) today. The winds are coming from the southwest at 15 to 25 mph. There is a 100% chance of rain.', response_metadata={'id': 'msg_01GRETGUHXrFd7rXCk6ut2yC', 'model': 'claude-3-opus-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 670, 'output_tokens': 62}}, id='run-f6d57c9c-d76a-4f0c-8f85-6b43c27a6324-0')